# Task One -- Sentiment Analysis (FiQA SA)

## Importing Packages for Part One

In [1]:
# from datasets import load_dataset # Doesn't work on
from sklearn.model_selection import train_test_split

import numpy as np
import json

ModuleNotFoundError: No module named 'datasets'

## Importing the Training Files

In [3]:
train_files = ["data_cache/FiQA_ABSA_task1/task1_headline_ABSA_train.json",
               "data_cache/FiQA_ABSA_task1/task1_post_ABSA_train.json"]

In [4]:
def load_fiqa_sa_from_json(json_files):
    train_text = []
    train_labels = []

    for file in json_files:
        with open(file, "r", encoding="utf-8") as handle:  # UTF-8 is needed to prevent a UnicodeDecodeError I was recieving
            dataf = json.load(handle)

        dataf_text = [dataf[k]["sentence"] for k in dataf.keys()]
        # print(len(dataf_text))
        train_text.extend(dataf_text)

        dataf_labels = [float(dataf[k]["info"][0]["sentiment_score"]) for k in dataf.keys()]
        # print(len(dataf_labels))
        train_labels.extend(dataf_labels)

    train_text = np.array(train_text)
    train_labels = np.array(train_labels)
    
    return train_text, train_labels


def threshold_scores(scores):
    """
    Convert sentiment scores to discrete labels.
    0 = negative.
    1 = neutral.
    2 = positive.
    """
    labels = []
    for score in scores:
        if score < -0.2:
            labels.append(0)
        elif score > 0.2:
            labels.append(2)
        else:
            labels.append(1)
            
    return np.array(labels)


all_text, all_labels = load_fiqa_sa_from_json(train_files)
    
print(f'Number of instances: {len(all_text)}')
print(f'Number of labels: {len(all_labels)}')

all_labels = threshold_scores(all_labels)
print(f'Number of negative labels: {np.sum(all_labels==0)}')
print(f'Number of neutral labels: {np.sum(all_labels==1)}')
print(f'Number of positive labels: {np.sum(all_labels==2)}')

Number of instances: 1111
Number of labels: 1111
Number of negative labels: 310
Number of neutral labels: 195
Number of positive labels: 606


## Creating the Test Split

In [5]:
# Split test data from training data
train_documents, test_documents, train_labels, test_labels = train_test_split(
    all_text, 
    all_labels, 
    test_size=0.2, 
    stratify=all_labels)  # make sure the same proportion of labels is in the test set and training set

# Split validation data from training data
train_documents, val_documents, train_labels, val_labels = train_test_split(
    train_documents, 
    train_labels, 
    test_size=0.15, 
    stratify=train_labels)  # make sure the same proportion of labels is in the test set and training set

print(f'Number of training instances = {len(train_documents)}')
print(f'Number of validation instances = {len(val_documents)}')
print(f'Number of test instances = {len(test_documents)}')

print(f'\nWhat does one instance look like from the training set?:\n{train_documents[234]}')
print(f'...and here is its corresponding sentiment label: {train_labels[234]}')

Number of training instances = 754
Number of validation instances = 134
Number of test instances = 223

What does one instance look like from the training set?:
$GS sells flat..I am out..
...and here is its corresponding sentiment label: 0


## Preprocessing the Data